<a href="https://colab.research.google.com/github/vishal7379/Colab/blob/main/NL_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import json

# ---------------- SCHEMAS ----------------
SCHEMAS = [
    {
        "tables": {
            "employees": ["id", "name", "salary", "dept_id"],
            "departments": ["id", "name"]
        },
        "numeric": {
            "employees": ["id", "salary", "dept_id"]
        },
        "text": {
            "employees": ["name"]
        },
        "join": ("employees", "departments", "dept_id", "id")
    },
    {
        "tables": {
            "students": ["id", "name", "marks", "class_id"],
            "classes": ["id", "name"]
        },
        "numeric": {
            "students": ["id", "marks", "class_id"]
        },
        "text": {
            "students": ["name"]
        },
        "join": ("students", "classes", "class_id", "id")
    }
]

AGGS = ["COUNT", "SUM", "AVG", "MAX", "MIN"]

# ---------------- GENERATOR ----------------
def generate_example():
    db = random.choice(SCHEMAS)
    schema = db["tables"]

    main_table = list(schema.keys())[0]
    cols = schema[main_table]

    intent = random.choices(
        ["SELECT", "AGG", "WHERE", "GROUP", "JOIN"],
        weights=[0.30, 0.25, 0.20, 0.15, 0.10]
    )[0]

    # ---------- SELECT ----------
    if intent == "SELECT":
        col = random.choice(cols)
        question = random.choice([
            f"show {col} of {main_table}",
            f"get {col} from {main_table}",
            f"list {col} in {main_table}",
            f"display {col} for {main_table}"
        ])
        sql = f"SELECT {col} FROM {main_table}"

    # ---------- AGGREGATION ----------
    elif intent == "AGG":
        agg = random.choice(AGGS)

        if agg == "COUNT":
            col = random.choice(cols)
        else:
            col = random.choice(db["numeric"][main_table])

        question = random.choice([
            f"show {agg.lower()} of {col} of {main_table}",
            f"what is the {agg.lower()} {col} in {main_table}",
            f"give me the {agg.lower()} {col} from {main_table}"
        ])
        sql = f"SELECT {agg}({col}) FROM {main_table}"

    # ---------- WHERE ----------
    elif intent == "WHERE":
        col = random.choice(db["numeric"][main_table])
        val = random.choice([10, 20, 50, 100])
        question = random.choice([
            f"show {col} of {main_table} where {col} > {val}",
            f"list {main_table} with {col} greater than {val}",
            f"get {col} from {main_table} having {col} above {val}"
        ])
        sql = f"SELECT {col} FROM {main_table} WHERE {col} > {val}"

    # ---------- GROUP BY ----------
    elif intent == "GROUP":
        agg = random.choice(["COUNT", "AVG"])
        group_col = random.choice(db["text"][main_table])
        num_col = random.choice(db["numeric"][main_table])

        question = random.choice([
            f"show {agg.lower()} of {num_col} per {group_col}",
            f"get {agg.lower()} {num_col} grouped by {group_col}",
            f"list {group_col} wise {agg.lower()} {num_col}"
        ])
        sql = (
            f"SELECT {group_col}, {agg}({num_col}) "
            f"FROM {main_table} GROUP BY {group_col}"
        )

    # ---------- JOIN ----------
    else:
        t1, t2, c1, c2 = db["join"]
        question = random.choice([
            f"show {t1} name and {t2} name",
            f"get {t1} names with their {t2}",
            f"list {t1} and corresponding {t2}"
        ])
        sql = (
            f"SELECT {t1}.name, {t2}.name "
            f"FROM {t1} JOIN {t2} ON {t1}.{c1} = {t2}.{c2}"
        )

    return {
        "question": question,
        "schema": schema,
        "sql": sql
    }

# ---------------- DATASET CREATION ----------------
def generate_dataset(n=50_000, outfile="nl2sql_varied.json"):
    data = []
    for i in range(n):
        data.append(generate_example())
        if (i + 1) % 10_000 == 0:
            print(f"Generated {i+1} examples")

    with open(outfile, "w") as f:
        json.dump(data, f, indent=2)

    print("✅ Dataset size:", len(data))


# 🔥 CHANGE n TO 300_000 IF YOU WANT
generate_dataset(n=50_000)


Generated 10000 examples
Generated 20000 examples
Generated 30000 examples
Generated 40000 examples
Generated 50000 examples
✅ Dataset size: 50000


In [3]:
# import json

# with open("nl2sql_varied.json", "r") as f:
#     dataset = json.load(f)

# print("Loaded dataset size:", len(dataset))

# from collections import Counter

# ENC_VOCAB = {"<PAD>":0, "<UNK>":1, "<SEP>":2}
# DEC_VOCAB = {"<PAD>":0, "<UNK>":1, "<BOS>":2, "<EOS>":3}


# def add(vocab, tok):
#     if tok not in vocab:
#         vocab[tok] = len(vocab)

# for ex in dataset:
#     # NL tokens
#     for t in ex["question"].lower().split():
#         add(ENC_VOCAB, t)

#     # Schema tokens
#     for t, cols in ex["schema"].items():
#         add(ENC_VOCAB, t)
#         for c in cols:
#             add(ENC_VOCAB, f"{t}.{c}")

#     # SQL tokens
#     for ex in dataset:
#       for t in ex["sql"].lower().split():
#           if t not in DEC_VOCAB:
#               DEC_VOCAB[t] = len(DEC_VOCAB)


# print("Encoder vocab:", len(ENC_VOCAB))
# print("Decoder vocab:", len(DEC_VOCAB))
import json

with open("nl2sql_varied.json", "r") as f:
    dataset = json.load(f)

print("Loaded dataset size:", len(dataset))

ENC_VOCAB = {"<PAD>":0, "<UNK>":1, "<SEP>":2}
DEC_VOCAB = {"<PAD>":0, "<UNK>":1, "<BOS>":2, "<EOS>":3}

def add(vocab, tok):
    if tok not in vocab:
        vocab[tok] = len(vocab)

for ex in dataset:
    # -------- Encoder vocab --------
    for t in ex["question"].lower().split():
        add(ENC_VOCAB, t)

    for table, cols in ex["schema"].items():
        add(ENC_VOCAB, table)
        for col in cols:
            add(ENC_VOCAB, f"{table}.{col}")

    # -------- Decoder vocab --------
    for t in ex["sql"].lower().split():
        add(DEC_VOCAB, t)

print("Encoder vocab:", len(ENC_VOCAB))
print("Decoder vocab:", len(DEC_VOCAB))


Loaded dataset size: 50000
Encoder vocab: 62
Decoder vocab: 62


In [4]:
import torch
from torch.utils.data import Dataset

class NL2SQLDataset(Dataset):
    def __init__(self, data, enc_vocab, dec_vocab,
                 max_src_len=80, max_tgt_len=60):
        self.data = data
        self.enc_vocab = enc_vocab
        self.dec_vocab = dec_vocab
        self.max_src_len = max_src_len
        self.max_tgt_len = max_tgt_len

    def __len__(self):
        return len(self.data)

    def encode_src(self, tokens):
        ids = [self.enc_vocab.get(t, self.enc_vocab["<UNK>"]) for t in tokens]
        ids = ids[:self.max_src_len]
        pad_len = self.max_src_len - len(ids)
        return ids + [self.enc_vocab["<PAD>"]] * pad_len

    def encode_tgt(self, tokens):
        ids = [self.dec_vocab["<BOS>"]] + \
              [self.dec_vocab.get(t, self.dec_vocab["<UNK>"]) for t in tokens] + \
              [self.dec_vocab["<EOS>"]]

        ids = ids[:self.max_tgt_len]
        pad_len = self.max_tgt_len - len(ids)
        return ids + [self.dec_vocab["<PAD>"]] * pad_len

    def __getitem__(self, idx):
        ex = self.data[idx]

        # -------- Encoder input (NL + schema) --------
        question_tokens = ex["question"].lower().split()

        schema_tokens = [
            f"{table}.{col}"
            for table, cols in ex["schema"].items()
            for col in cols
        ]

        src_tokens = question_tokens + ["<SEP>"] + schema_tokens
        src_ids = self.encode_src(src_tokens)

        # -------- Decoder target (SQL) --------
        sql_tokens = ex["sql"].lower().split()
        tgt_ids = self.encode_tgt(sql_tokens)

        return (
            torch.tensor(src_ids, dtype=torch.long),
            torch.tensor(tgt_ids, dtype=torch.long)
        )


In [5]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, dim_ff=1024, num_layers=4):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, d_model, padding_idx=0)
        layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_ff,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(layer, num_layers)

    def forward(self, x):
        # x: [B, T]
        pad_mask = (x == 0)        # True where PAD
        emb = self.emb(x)
        return self.encoder(emb, src_key_padding_mask=pad_mask)

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, dim_ff=1024, num_layers=4):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, d_model)
        layer = nn.TransformerDecoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_ff,
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(layer, num_layers)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, y, memory, memory_pad_mask):
        # y: [B, T]
        tgt_len = y.size(1)

        # causal mask (prevent seeing future tokens)
        causal_mask = torch.triu(
            torch.ones(tgt_len, tgt_len, device=y.device),
            diagonal=1
        ).bool()

        emb = self.emb(y)
        out = self.decoder(
            emb,
            memory,
            tgt_mask=causal_mask,
            memory_key_padding_mask=memory_pad_mask
        )
        return self.fc(out)


In [7]:
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize models (PASS vocab sizes)
enc = Encoder(vocab_size=len(ENC_VOCAB)).to(device)
dec = Decoder(vocab_size=len(DEC_VOCAB)).to(device)

# DataLoader (NO collate_fn)
loader = DataLoader(
    NL2SQLDataset(
        data=dataset,
        enc_vocab=ENC_VOCAB,
        dec_vocab=DEC_VOCAB,
        max_src_len=80,
        max_tgt_len=60
    ),
    batch_size=64,
    shuffle=True
)

# Optimizer
opt = optim.Adam(
    list(enc.parameters()) + list(dec.parameters()),
    lr=1e-4
)

# Loss (ignore PAD in decoder vocab)
loss_fn = nn.CrossEntropyLoss(ignore_index=DEC_VOCAB["<PAD>"])

print("🚀 Training...")
for epoch in range(5):
    enc.train()
    dec.train()
    total = 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        opt.zero_grad()

        # Encoder
        mem = enc(x)
        src_pad_mask = (x == 0)

        # Decoder (teacher forcing)
        out = dec(
            y=y[:, :-1],
            memory=mem,
            memory_pad_mask=src_pad_mask
        )

        # Loss
        loss = loss_fn(
            out.reshape(-1, len(DEC_VOCAB)),
            y[:, 1:].reshape(-1)
        )

        loss.backward()
        opt.step()

        total += loss.item()

    print(f"Epoch {epoch+1} | Loss {total / len(loader):.4f}")


🚀 Training...
Epoch 1 | Loss 0.1452
Epoch 2 | Loss 0.0024
Epoch 3 | Loss 0.0010
Epoch 4 | Loss 0.0005
Epoch 5 | Loss 0.0003


In [10]:
inv_dec_vocab = {v: k for k, v in DEC_VOCAB.items()}

def infer_sql(question, schema, max_len=50):
    enc.eval()
    dec.eval()

    # Build encoder tokens
    tokens = question.lower().split() + ["<SEP>"] + [
        f"{t}.{c}" for t, cols in schema.items() for c in cols
    ]

    x = torch.tensor([
        ENC_VOCAB.get(t, ENC_VOCAB["<UNK>"]) for t in tokens
    ]).unsqueeze(0).to(device)

    # Encoder
    with torch.no_grad():
        memory = enc(x)
        memory_pad_mask = (x == ENC_VOCAB["<PAD>"])

        # Decoder autoregressive loop
        y = torch.tensor([[DEC_VOCAB["<BOS>"]]], device=device)

        for _ in range(max_len):
            logits = dec(
                y=y,
                memory=memory,
                memory_pad_mask=memory_pad_mask
            )

            next_token = logits[:, -1].argmax(dim=-1)
            y = torch.cat([y, next_token.unsqueeze(1)], dim=1)

            if next_token.item() == DEC_VOCAB["<EOS>"]:
                break

    # Decode tokens to SQL
    sql_tokens = [
        inv_dec_vocab[t.item()]
        for t in y[0][1:]
        if t.item() not in (
            DEC_VOCAB["<EOS>"],
            DEC_VOCAB["<PAD>"]
        )
    ]

    return " ".join(sql_tokens)


In [17]:
schema = {
    "employees": ["id","name","salary","dept_id"],
    "departments": ["id","name"]
}

print(infer_sql("give names of employees", schema))
print(infer_sql("show name of employees where salary is greater than 50", schema))


select name from employees
select salary from employees where salary > 50
